# Набор данных

[Набор данных](https://drive.google.com/file/d/1qCMvjO0GlNk1WCu7HKn5W8vUNVdJ7eZD/view?usp=share_link) -- запись 2.2 часов игры в простую мини игру obstacles, целью которой является преодоление случайных препятствий (см. [видео](https://drive.google.com/file/d/1aUN1821DvO3V1zxNGBBkSVJ6e93_SxuZ/view?usp=share_link) для примера). Состояние игры сохранялось каждые 50 мс (1 игровой тик).

Признаки:
- поставил ли игрок красный блок в этот игровой тик (`placed_block`)
- держит ли игрок блок (`holds_block`)
- координаты игрока (`x`, `y`, `z`)
- угол поворота камеры относительно осей координат в градусах (`yaw`, `pitch`)
- может ли игрок летать (`can_fly`)
  - `true` во время возрождения
- координаты блока, который игрок сейчас может поставить (`x_block`, `y_block`, `z_block`)
  - `NaN`, если игрок не смотрит на блок
- упал ли игрок только что в бездну (`missclick`)
- какие клавиши нажаты (`key_forward`, `key_back`, `key_left`, `key_right`, `key_jump`, `key_sneak`)
- стою ли игрок на блоке (`on_ground`)
- находятся ли блоки в кубе 5x5x5 блоков вокруг игрока (`player[x][y][z]`, где $x, y, z \in \{-2, -1, 0, 1, 2\}$)
- находятся ли блоки в кубе 5x5x5 блоков вокруг блока, на который смотрит игрок (`block[dx][dy][dz]`, где $x, y, z \in \{-2, -1, 0, 1, 2\}$)
  - если игрок не смотрит на блок, то используется блок на расстоянии 4 блоков от него в направлении взгляда
- находятся ли блоки в кубе 5x5x5 блоков вокруг средней точки между центрами кубов из двух предыдущих пунктов (`middle[dx][dy][dz]`, где $x, y, z \in \{-2, -1, 0, 1, 2\}$)

In [ ]:
!pip install pandas==1.5.3
!pip install matplotlib==3.7.1
!pip install seaborn==0.12.2

In [2]:
import pandas as pd
import numpy as np

# Разведочный анализ (EDA) и преобразования данных

Загрузим частично предобработанный в предыдущем задании набор данных

In [3]:
df = pd.read_csv('prepared_place.csv')

Посмотрим на средние значения и стандартные отклонения признаков, заметим, что стандартное отклонение сильно меняется от признака к признаку: от $0.09$ для `vel_z` до $52$ для `yaw`. Необходима нормализация.

In [4]:
pd.DataFrame({"Mean": df.mean(), "Std": df.std()}).sort_values(by=["Std"])

,Mean,Std
vel_z,0.003975,0.092472
middle[0][1][0],0.008628,0.092489
block[0][1][0],0.009209,0.095521
middle[1][1][0],0.009518,0.097098
vel_x,-0.214484,0.119356
...,...,...
rel_block_y,-0.954000,0.850004
y,104.280098,2.374035
z,1989.270528,2.783446
pitch,57.743340,14.661339


Нормализуем признаки

In [5]:
df = (df - df.mean()) / df.std()

Разобьём признаки на массивы и не массивы

In [6]:
arr_columns = [col for col in df.columns if '[' in col]
non_arr_columns = [col for col in df.columns if '[' not in col]
arr_to_non_arr_ratio = len(arr_columns) / len(non_arr_columns)
print("Array columns:", len(arr_columns))
print("Non array columns:", len(non_arr_columns))
print("Array to non array columns ratio:", arr_to_non_arr_ratio)

Array columns: 322
Non array columns: 23
Array to non array columns ratio: 14.0


Заметим, что признаков-массивов в 14 раз больше, однако из экспертизы в предметной области известно, что они не столь информативны, поэтому отмасштабируем их

In [7]:
df[arr_columns] /= arr_to_non_arr_ratio

Также известно, что блоки ближе к центру более важные, поэтому отмасштабируем признаки-массивы ещё раз в соответствии с нормой их индексов

In [8]:
importances = [1 / (1 + sum(int(cord) * int(cord) for cord in col.replace("]", "").split("[")[1:])) for col in arr_columns]

importances /= np.mean(importances)

for col, importance in zip(arr_columns, importances):
  df[col] *= importance

Ещё раз посмотрим на средние значения и стандартные отклонения признаков, чтобы убедиться, что всё отмасштабировано правильно

In [9]:
pd.DataFrame({"Mean": df.mean(), "Std": df.std()}).sort_values(by=["Std"])

,Mean,Std
middle[2][-2][-2],-4.020773e-18,0.030296
middle[-2][2][2],1.219978e-18,0.030296
middle[-2][-2][2],1.718279e-18,0.030296
player[-2][2][-2],-6.873116e-20,0.030296
block[2][2][2],0.000000e+00,0.030296
...,...,...
y_frac,-2.100424e-16,1.000000
vel_x,3.678492e-16,1.000000
will_place,7.477950e-17,1.000000
key_back,-2.859216e-17,1.000000


Сохраним обработанный набор данных

In [10]:
df.to_csv("normalized_prepared_place.csv", index=False)